In [1]:
import json
import pickle

# Load the provided JSON file
with open('train_data.json', 'r') as file:
    train_data = json.load(file)

# Save the data into a pickle file
pkl_path = 'train_data.pkl'
with open(pkl_path, 'wb') as pkl_file:
    pickle.dump(train_data, pkl_file)

pkl_path


'train_data.pkl'

In [2]:
train_data = pickle.load(open('train_data.pkl', 'rb'))
train_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [3]:
# nlp = spacy.blank('en')

# def train_model(train_data):
#     if 'ner' not in nlp.pipe_names:
#         # ner = nlp.create_pipe('ner')
#         ner = nlp.add_pipe('ner', last = True)
    
#     for _, annotation in train_data:
#         for ent in annotation['entities']:
#             ner.add_label(ent[2])
            
    
#     other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
#     with nlp.disable_pipes(*other_pipes):  # only train NER
#         optimizer = nlp.begin_training()
#         for itn in range(10):
#             print("Statring iteration " + str(itn))
#             random.shuffle(train_data)
#             losses = {}
#             index = 0
#             for text, annotations in train_data:
#                 try:
#                     nlp.update(
#                         [text],  # batch of texts
#                         [annotations],  # batch of annotations
#                         drop=0.2,  # dropout - make it harder to memorise data
#                         sgd=optimizer,  # callable to update weights
#                         losses=losses)
#                 except Exception as e:
#                     pass
                
#             print(losses)
    

In [4]:
def check_alignment(train_data):
    for text, annotation in train_data:
        doc = nlp.make_doc(text)
        entities = annotation['entities']
        tags = offsets_to_biluo_tags(doc, entities)
        if '-' in tags:
            print("Misaligned entities found:")
            print("Text:", text[:50], "...")
            for start, end, label in entities:
                span_text = text[start:end]
                print(f" Entity: {span_text} ({start}, {end}, {label})")
            print()

In [5]:
def clean_overlapping_entities(train_data):
    cleaned_data = []
    for text, annotations in train_data:
        entities = annotations['entities']
        entities = sorted(entities, key=lambda x: x[0])  # sort by start index
        
        non_overlapping_entities = []
        prev_end = -1
        for start, end, label in entities:
            if start >= prev_end:
                non_overlapping_entities.append((start, end, label))
                prev_end = end
            else:
                # Skip overlapping entities
                print(f"Removed overlapping entity: {(start, end, label)}")
                
        cleaned_data.append((text, {'entities': non_overlapping_entities}))
    return cleaned_data




In [7]:
import spacy
import pickle
import random
from spacy.training import Example

nlp = spacy.blank('en')

train_data = pickle.load(open('train_data.pkl', 'rb'))
train_data = clean_overlapping_entities(train_data)
# check_alignment(train_data)


def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.add_pipe('ner', last=True)
    else:
        ner = nlp.get_pipe('ner')

    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(10):
            print(f"Starting iteration {itn}")
            random.shuffle(train_data)
            losses = {}

            examples = []
            for text, annotations in train_data:
                doc = nlp.make_doc(text)
                examples.append(Example.from_dict(doc, annotations))

            nlp.update(
                examples,
                drop=0.2,
                sgd=optimizer,
                losses=losses
            )

            print(losses)



Removed overlapping entity: (1209, 1215, 'Companies worked at')
Removed overlapping entity: (1417, 1423, 'Companies worked at')
Removed overlapping entity: (1696, 1702, 'Companies worked at')
Removed overlapping entity: (1749, 1755, 'Companies worked at')
Removed overlapping entity: (34, 49, 'Companies worked at')
Removed overlapping entity: (4186, 4191, 'Companies worked at')
Removed overlapping entity: (2128, 2143, 'Companies worked at')
Removed overlapping entity: (894, 897, 'Skills')
Removed overlapping entity: (1153, 1183, 'College Name')
Removed overlapping entity: (1742, 1745, 'Skills')
Removed overlapping entity: (1765, 1771, 'Skills')
Removed overlapping entity: (1811, 1815, 'Skills')
Removed overlapping entity: (1844, 1860, 'Skills')
Removed overlapping entity: (3535, 3541, 'Companies worked at')
Removed overlapping entity: (3714, 3720, 'Companies worked at')
Removed overlapping entity: (6861, 6870, 'Companies worked at')
Removed overlapping entity: (203, 207, 'Skills')
Remov

In [8]:
# Call the training function
train_model(train_data)


Starting iteration 0


/Users/sharyl/Desktop/capstone2/.venv/lib/python3.9/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Palani S Senior Technology Support Executive at In..." with entities "[(0, 8, 'Name'), (9, 44, 'Designation'), (48, 55, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/sharyl/Desktop/capstone2/.venv/lib/python3.9/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Sai Dhir - Email me on Indeed: indeed.com/r/Sai-Dh..." with entities "[(0, 8, 'Name'), (31, 69, 'Email Address'), (88, 1...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/sharyl/Desktop/capstone2/.venv/lib/python3.9/site-packages/sp

{'ner': np.float32(125049.3)}
Starting iteration 1
{'ner': np.float32(124099.74)}
Starting iteration 2
{'ner': np.float32(122865.0)}
Starting iteration 3
{'ner': np.float32(121385.66)}
Starting iteration 4
{'ner': np.float32(118875.1)}
Starting iteration 5
{'ner': np.float32(113966.76)}
Starting iteration 6
{'ner': np.float32(106265.625)}
Starting iteration 7
{'ner': np.float32(89003.54)}
Starting iteration 8
{'ner': np.float32(59878.64)}
Starting iteration 9
{'ner': np.float32(34284.85)}


In [9]:
nlp.to_disk('nlp_model')

In [10]:
nlp_model = spacy.load('nlp_model')

In [11]:
train_data[0][0]

"Zaheer Uddin Technical Project Manager  Hyderabad, Telangana - Email me on Indeed: indeed.com/r/Zaheer-Uddin/fd9892e91ac9a58f  To be associated with a dynamic team in a progressive organization that gives me the scope to apply and enrich my knowledge and skills with continual learning and professional growth.  WORK EXPERIENCE  Technical Project Manager  DELL EMC -  August 2015 to May 2018  Provide expertise to support the project team in the following areas: * Activity and resource planning. * Analyzing and managing project risk. * Planning, organizing, and Leading the Project teams from the entire metrics standpoint. * Scope &amp; Charter Management. * Handled the Account Level Audit (ERM, ITIL) * Estimations and cost management. * Project Timeline planning and management * Procurement, Software Asset management, Change management and Configuration management. * Issue resolution and escalation * Communications &amp; status reporting * Scheduling, attending and running all project mee

In [ ]:
import fitz
fname = 'Alice Clark CV.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
    text = text + str(page.get_text())

tx = " ".join(text.split('\n')) 
print(tx)

Alice Clark  AI / Machine Learning    Delhi, India Email me on Indeed  •  20+ years of experience in data handling, design, and development  •  Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to  data warehousing and business intelligence  •  Database: Experience in database designing, scalability, back-up and recovery, writing and  optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.  Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure,  Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake  analytics(U-SQL)  Willing to relocate anywhere    WORK EXPERIENCE  Software Engineer  Microsoft – Bangalore, Karnataka  January 2000 to Present  1. Microsoft Rewards Live dashboards:  Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping  online. Microsoft Rewards members can earn points when searching with Bing, bro

In [ ]:
# test_text = "John Doe works at Google."
# doc_test = nlp_model(test_text)
# print([(ent.label_, ent.text) for ent in doc_test.ents])

[]


In [14]:
doc = nlp_model(tx)
print(doc.ents)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

()


In [ ]:
import spacy
import fitz  # PyMuPDF (make sure you installed via `pip install PyMuPDF`)

# 1) Load your custom spaCy model
#    Replace "my_custom_model" with the actual path or name of your trained model.
nlp_model = spacy.load("en_core_web_sm")

# 2) Extract text from PDF using PyMuPDF
pdf_path = "Alice Clark CV.pdf"
doc_pdf = fitz.open(pdf_path)

extracted_text = ""
for page in doc_pdf:
    # For PyMuPDF >= 1.19.0 use get_text()
    extracted_text += page.get_text()

# (Optional) Clean or preprocess text if needed
# e.g., remove extra newlines, etc.
# extracted_text = " ".join(extracted_text.split())

# 3) Apply the model to the extracted text
doc_spacy = nlp_model(extracted_text)

# 4) Print recognized entities
print("\nRecognized Entities:")
for ent in doc_spacy.ents:
    print(f"{ent.label_.upper():<25} {ent.text}")


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.